<div style='float: right'><img src='pic/suiri.png'/></div>

## <div id='suiri' />推理パズル

In [ ]:
#!pip3 install pulp ortoolpy
import pandas as pd
from pulp import LpProblem, lpSum, value
from itertools import chain, product
from ortoolpy import addbinvar
kinds = [['明','勇','正','洋'],
         ['傘','靴','紙','糸'],
         ['赤','青','白','黒']]
data = [s.split(',') for s in """\
1,明,,白
0,明,糸,
1,,紙,青
0,勇,紙,
1,正,靴,
0,正,,赤""".splitlines()]

### 問題
* 3つの組(kinds)を入力し、各組間の対応を求めます
* ヒント(data)の意味
    * 明は白いのを買った
    * 明は糸じゃない
    * 青い紙を買った人がいる
    * 勇は紙じゃない
    * 正は靴を買った
    * 正は赤じゃない

### 変数
* $v_{ijk}$: 表iの位置(j,k) (1)

### 制約
* 縦横で丸は1つ (2)
* AかつB、BかつCなら、CかつA (3)
* ヒントを満たすこと (4)

In [ ]:
a1 = pd.DataFrame((s0,s1,'',addbinvar()) for s0,s1 in product(kinds[0],kinds[1]))
a2 = pd.DataFrame((s0,'',s2,addbinvar()) for s0,s2 in product(kinds[0],kinds[2]))
a3 = pd.DataFrame(('',s1,s2,addbinvar()) for s1,s2 in product(kinds[1],kinds[2]))
a = pd.concat([a1,a2,a3])
a1.columns = a2.columns = a3.columns = a.columns = ['人','物','色','Var']
a[:2]

In [ ]:
m = LpProblem()
for a0,c1,c2 in [(a1,'人','物'),(a2,'人','色'),(a3,'物','色')]:
    for _,v in chain(a0.groupby(c1),a0.groupby(c2)):
        m += lpSum(v.Var) == 1
for s1,s2,s3 in product(*kinds):
    vlst = [a1.query(f'人=="{s1}"&物=="{s2}"').Var.iloc[0],
            a2.query(f'人=="{s1}"&色=="{s3}"').Var.iloc[0],
            a3.query(f'物=="{s2}"&色=="{s3}"').Var.iloc[0]]
    for v in vlst:
        m += lpSum(vlst) <= 1+2*v
for c,s1,s2,s3 in data:
    m += a.query(f'人=="{s1}"&物=="{s2}"&色=="{s3}"').Var.iloc[0] == int(c)
m.solve()

In [ ]:
a1['Val'] = a1.Var.apply(value)
a2['Val'] = a2.Var.apply(value)
a1.loc[a1.Val>0.5,['人','物']].merge(a2.loc[a2.Val>0.5,['人','色']])